# Лаба 8. Мониторинг качества работы модели машинного обучения с использованием дэшбордов

Итак, у нас есть модель и предсказания пола и возрастной группы по ней. Мы можем посчитать метрику модели, зная метки тестового датасета. Однако в проме у нас нет истинных меток, и надо как-то понимать, модель дает нам то качество предсказаний, на которое мы расчитываем? Не деградирует ли она со временем?

К этой проблеме подходят по разному.

1. Бизнес метрика более высокого уровня.

Мы не может посчитать точность предсказания модели, но мы можем посчитать эффект применения модели. Например, мы определяем пол и возраст новых пользователей, чтобы сделать им таргетированное предложение в рамках маркетинговой кампании. Тогда мы можем подсчитать эффективность кампании с таргетированием через коэффициент конверсии, то есть сколько людей приняло предложение. Конечно, наша модель тут только один из факторов успеха кампании, но все же, стат. методами мы сможем оценить вклад и нашей модели. Например, можем сравнивать конверсию клиентов таргетированной рекламы с полностью случайной рекламной рассылкой. Или сравнивания с разультатами работы другой модели.

2. А/Б тестирование

Мы сравниваем работу модели на двух группах: одна – обычные рандомные пользователи, другая – контрольная – состоит из пользователей, которых мы знаем.

3. Смотрим на внутренние показатели работы модели.

К таковым относится, например, распределение предсказаний модели по классам. Если оно начинает сдвигаться со временем, то возможно, что надо переучивать модель на свежих данных.

## I. Задача с высоты птичьего полета

Давайте проиллюстрируем последний способ. Вы запишите ваши предсказания в ElasticSearch и далее построите dashboard с графиками изменений числа предсказаний со временем. По линиям тренда будет видно, дрейфует ли модель.

## II. Реализация

Возмите 5000 предсказаний, которые у вас есть после выполнения Лабы 7. Разбейте их на 25 частей. Будем предполагать, что каждая часть – предсказания за определенный день, начиная с 1-го июня 2020. 

Измените ваше приложение из Лабы 7а таким образом, чтобы записывать предсказания в индекс Elasticsearch под названием `name_surname_lab08`. Помимо предсказаний, в сообщениях также должно содержаться поле `date` с временной меткой в миллисекундах эпохи, назначенной в соответствии с описанным выше – то есть первым 200 сообщениям должна назначаться метка 1-го июня, следующим – 2-го июня и т.д. Перед тем как записывать события в Elastic, создайте индекс используя REST API. Смотрите короткую [справку по АПИ Elasticsearch и Kibana](Elastic_API.md).

Постройте в Кибане график (visualization) с числом предсказаний каждого класса в предсказаниях в зависимости от времени.  

Для этого перейдите в Kibana по адресу 10.0.1.9:5601, в пункте меню Visualization выберите тип `timelion`, и в качестве `timelion expression` введите: `.es(index=name_surname_lab08, metric=count, timefield=date, split=gender_age:10)`/ Сохраните под именем `name_surname_lab08`.

Посмотрите, меняется ли распределение классов со временем? 

График должен называться `name_surname_lab08` и выглядеть примерно так:

![lab08.png](img/lab08.png)

Далее постройте график с трендами придсказаний каждого класса. Воспользуйтесь `.trend()`. Повторите процедуру, только добавьте к концу выражения `.trend()`. Этот график должен называться `name_surname_lab08_trend` и выглядеть он должен примерно так:

![lab08_trend.png](img/lab08_trend.png)

Посмотрите, есть ли тренды? В качестве самостоятельного задания подсчитайте статистическими методами, можно ли сделать вывод о наличии тренда? Т.е. примите в качестве нулевой гипотезы, что тренда нет, а в качестве альтернативной, что он есть. Посчитайте p-value.

Далее создайте dashboard в пункте меню `Dashboard` под названием `name_surname_lab08` и добавьте туда оба графика.

## III. Оформление работы

Ваш проект в репо в подпапке lab08 должен называться `model_quality`.

## IV. Доступ к Elastic и Kibana

* Elasticsearch REST API: 10.0.1.9:9200
* Kibana Web UI, REST API: 10.0.1.9:5601 

Для логина в Web UI и аутентикации REST API используйте ваш логин и пароль в ЛК. Web UI доступен с пробросом порта по туннелю или через socks-прокси. Авторизация аккаунтов настроена таким образом, что вы можете создавать индексы с шаблоном name_surname*, и не имеете доступа ни к каким другим индексам.

## V. Проверка

Чекер найдет ваш dashboard, скачает его в формате `json`, и проверит.

### Поля чекера

* `git_correct` – проверка репо
* `git_errors` – ошибки репо
* `index_correct` – в Elasticsearch имеется индекс `name_surname_lab08` с правильными полями
* `dashboard_correct` – в Kibana имеется dashboard `name_surname_lab08` и он правильный.
* `lab_result`

### Cамопроверка

#### Поиск dashboard

`curl -X GET 10.0.1.9:5601/api/saved_objects/_find?type=dashboard&search=artem_trunov_lab08`

#### Считывание dashboard

`curl -X GET 10.0.1.9:5601/api/kibana/dashboards/export?dashboard=d818cd30-a985-11ea-8889-8de7ce1ad0f9`

В вашем dashboard должно быть три объекта - один с типом dashboard и названием name_surname_lab08, и два с типом visualization и названиями name_surname_lab08, name_surname_lab08_trend.



In [ ]:
PUT _template/kirill_likhouzov_lab08
{
  "index_patterns": ["kirill_likhouzov_lab08-*"],
  "settings": {
    "number_of_shards": 1,
    "number_of_replicas" : 1
  },
  "mappings": {
    "_doc": {
      "dynamic": true,
      "_source": {
        "enabled": true
      },
      "properties": {
        "uid": {
        "type": "keyword",
        "null_value": "NULL"
          },
          "gender_age": {
            "type": "keyword"
          },
          "date": {
            "type": "date"
          }
      }
    }
  }
}

In [5]:
import sys.process._
"cat /tmp/qwi7".!!

"{"index": {"_index": "artem_trunov_lab08", "_type": "_doc", "_id": 0}}
{"gender_age":"F:18-24","uid":"0000e7ca-32e6-4bef-bdca-e21c025071ff","date":1590969600000}
{"index": {"_index": "artem_trunov_lab08", "_type": "_doc", "_id": 1}}
{"gender_age":"M:45-54","uid":"009acdaa-e72d-46c3-b1a5-1d8e06781594","date":1590969600000}
{"index": {"_index": "artem_trunov_lab08", "_type": "_doc", "_id": 2}}
{"gender_age":"M:25-34","uid":"009bb5dd-3468-42f8-a2b9-03f223c63c1f","date":1590969600000}
{"index": {"_index": "artem_trunov_lab08", "_type": "_doc", "_id": 3}}
{"gender_age":"M:35-44","uid":"009bc013-cf60-40cc-b985-290a4ba5b504","date":1590969600000}
{"index": {"_index": "artem_trunov_lab08", "_type": "_doc", "_id": 4}}
{"gender_age":"M:25-34","uid":"009e8603-...


In [1]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.{SQLContext, DataFrame}
import org.apache.spark.sql.functions._

val conf = new SparkConf()
                            .setAppName("lab08")

val sparkSession = SparkSession.builder()
  .config(conf=conf)
  .getOrCreate()

var sc = sparkSession.sparkContext
val sqlContext = new SQLContext(sc)

conf = org.apache.spark.SparkConf@665e96ff
sparkSession = org.apache.spark.sql.SparkSession@a4ac577
sc = org.apache.spark.SparkContext@2aa21eeb
sqlContext = org.apache.spark.sql.SQLContext@3bbdbc58


org.apache.spark.sql.SQLContext@3bbdbc58

In [4]:
val dataset = spark
    .read
    .json("file:///tmp/qwi7")
    .filter($"index".isNull)
    .select($"uid", $"gender_age", $"date")
    .repartition(1)

dataset = [uid: string, gender_age: string ... 1 more field]


[uid: string, gender_age: string ... 1 more field]

In [5]:
dataset.show(3)

+--------------------+----------+-------------+
|                 uid|gender_age|         date|
+--------------------+----------+-------------+
|0000e7ca-32e6-4be...|   F:18-24|1590969600000|
|009acdaa-e72d-46c...|   M:45-54|1590969600000|
|009bb5dd-3468-42f...|   M:25-34|1590969600000|
+--------------------+----------+-------------+
only showing top 3 rows



In [6]:
%AddJar file:///data/home/kirill.likhouzov/Drivers/elasticsearch-spark-20_2.11-7.6.2.jar

Starting download from file:///data/home/kirill.likhouzov/Drivers/elasticsearch-spark-20_2.11-7.6.2.jar
Finished download of elasticsearch-spark-20_2.11-7.6.2.jar


In [8]:
import org.apache.spark.sql.functions._

val esOptions = 
    Map(
        "es.nodes" -> "10.0.1.9:9200/kirill_likhouzov_lab08", 
        "es.batch.write.refresh" -> "false",
        "es.nodes.wan.only" -> "true"   
    )

dataset
    .write
    .format("org.elasticsearch.spark.sql")
    .options(esOptions)
    .save("kirill_likhouzov_lab08-{date}/_doc")

esOptions = Map(es.nodes -> 10.0.1.9:9200/kirill_likhouzov_lab08, es.batch.write.refresh -> false, es.nodes.wan.only -> true)


Map(es.nodes -> 10.0.1.9:9200/kirill_likhouzov_lab08, es.batch.write.refresh -> false, es.nodes.wan.only -> true)

In [9]:
val esDf = spark.read.format("es").options(esOptions).load("kirill_likhouzov_lab08-*")
esDf.printSchema
esDf.show(1, 200, true)

root
 |-- date: timestamp (nullable = true)
 |-- gender_age: string (nullable = true)
 |-- uid: string (nullable = true)

-RECORD 0------------------------------------------
 date       | 2020-06-14 03:00:00                  
 gender_age | M:35-44                              
 uid        | 8a0213c8-a895-4809-a63d-5ad36e2d9faf 
only showing top 1 row



esDf = [date: timestamp, gender_age: string ... 1 more field]


[date: timestamp, gender_age: string ... 1 more field]

In [ ]:
var df = spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", KafkaService.bootstrapServers)
        .option("enable.auto.commit", KafkaService.enableAutoCommit)
        .option("failOnDataLoss", KafkaService.failOnDataLoss)
        .option("startingOffsets", KafkaService.startingOffsets)
        .option("subscribe", topicName)
        .option("group.id", groupId)
        .load()
    
    df.writeStream
    .outputMode(OutputMode.Append) //Only mode for ES
    .format("") //es
    .queryName("ElasticSink" + topicName)
    .start(indexName + "/broadcast") //ES index

In [11]:
sc.stop

lastException: Throwable = null
